In [1]:
%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d
from data4d import Data4d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Load time-lapse data...

In [2]:
filenames = ['/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t2.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t3.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t4.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t5.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t6.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t7.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t8.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t9.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t10.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t11.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t12.TIF']

In [3]:
data = Data4d( filenames, pixelsize=(1.,1.,5.), silent=False )
data.set_seg_params(K=30,max_delta_k=4)
objects = []
ecells = []

Dimensions of frame 0 :  (31, 512, 512)
Dimensions of frame 1 :  (31, 512, 512)
Dimensions of frame 2 :  (31, 512, 512)
Dimensions of frame 3 :  (31, 512, 512)
Dimensions of frame 4 :  (31, 512, 512)
Dimensions of frame 5 :  (31, 512, 512)
Dimensions of frame 6 :  (31, 512, 512)
Dimensions of frame 7 :  (31, 512, 512)
Dimensions of frame 8 :  (31, 512, 512)
Dimensions of frame 9 :  (31, 512, 512)
Dimensions of frame 10 :  (31, 512, 512)
Dimensions of frame 11 :  (31, 512, 512)


In [4]:
max_rs = (data.images[0].shape[2] / 8,
          data.images[0].shape[1] / 8,
          data.images[0].shape[0] / 2)

min_rs = (10,10,2)

## adding E cells

In [5]:
Ea = data.init_object("Ea")
objects.append(Ea)
ecells.append(Ea)

Ep = data.init_object("Ep")
objects.append(Ep)
ecells.append(Ep)

In [6]:
# data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), frame_to=11, seed_to=(105,310,16), segment_it=True )
data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), segment_it=True )
data.track(Ea, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ea" in frame 0 with seed coordinates [ 110.  300.   12.]
      Optimum energy:  94916.0
      Nodes in/out:  6602 8758
Added appearance for "Ea" in frame 1 with seed coordinates [ 110.  300.   12.]
      Optimum energy:  94323.0
      Nodes in/out:  6722 8638
    Updated center to [110 301  12]
Added appearance for "Ea" in frame 2 with seed coordinates [ 110.  301.   12.]
      Optimum energy:  83732.0
      Nodes in/out:  6861 8499
    Updated center to [110 301  12]
Added appearance for "Ea" in frame 3 with seed coordinates [ 110.  301.   12.]
      Optimum energy:  78480.0
      Nodes in/out:  7137 8223
    Updated center to [111 302  12]
Added appearance for "Ea" in frame 4 with seed coordinates [ 111.  302.   12.]
      Optimum energy:  77295.0
      Nodes in/out:  7152 8208
    Updated center to [113 302  12]
Added appearance for "Ea" in frame 5 with seed coordinates [ 113.  302.   12.]
      Optimum energy:  78002.0
      Nodes in/out:  6926 8434
    Update

In [8]:
data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), frame_to=11, seed_to=(140,360,16), segment_it=True )
# data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), segment_it=True )
# data.track(Ep, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ep" in frame 0 with seed coordinates [ 110.  350.   12.]
      Optimum energy:  86572.0
      Nodes in/out:  6047 9313
Added appearance for "Ep" in frame 1 with seed coordinates [ 112.  351.   12.]
      Optimum energy:  86738.0
      Nodes in/out:  6231 9129
Added appearance for "Ep" in frame 2 with seed coordinates [ 115.  352.   13.]
      Optimum energy:  82747.0
      Nodes in/out:  6778 8582
Added appearance for "Ep" in frame 3 with seed coordinates [ 118.  352.   13.]
      Optimum energy:  82826.0
      Nodes in/out:  6490 8870
Added appearance for "Ep" in frame 4 with seed coordinates [ 120.  353.   13.]
      Optimum energy:  80383.0
      Nodes in/out:  6177 9183
Added appearance for "Ep" in frame 5 with seed coordinates [ 122.  354.   14.]
      Optimum energy:  77167.0
      Nodes in/out:  6312 9048
Added appearance for "Ep" in frame 6 with seed coordinates [ 125.  355.   14.]
      Optimum energy:  76830.0
      Nodes in/out:  6307 9053
Added appeara

## adding direct neighbors

In [9]:
neighbors = []
for i in range(12):
    n = data.init_object("n"+str(i))
    objects.append(n)
    neighbors.append(n)

In [10]:
nseeds = [( 90,390,12),
          (150,410,12),
          (200,390,12),
          (190,315,12),
          (145,265,12),
          (110,230,12),
          ( 40,255,12),
          ( 45,300,18),
          ( 67,355,18),
          (110,275,24),
          (130,315,24),
          (125,360,24)]
for i in range(len(nseeds)):
    data.add_object_at( neighbors[i], min_rs, max_rs, frame=0, seed=nseeds[i], segment_it=True )
    data.track(neighbors[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "n0" in frame 0 with seed coordinates [  90.  390.   12.]
      Optimum energy:  75399.0
      Nodes in/out:  3969 11391
Added appearance for "n0" in frame 1 with seed coordinates [  90.  390.   12.]
      Optimum energy:  74563.0
      Nodes in/out:  4067 11293
    Updated center to [ 90 387  12]
Added appearance for "n0" in frame 2 with seed coordinates [  90.  387.   12.]
      Optimum energy:  73587.0
      Nodes in/out:  4201 11159
    Updated center to [ 90 384  12]
Added appearance for "n0" in frame 3 with seed coordinates [  90.  384.   12.]
      Optimum energy:  72912.0
      Nodes in/out:  4262 11098
    Updated center to [ 89 381  11]
Added appearance for "n0" in frame 4 with seed coordinates [  89.  381.   11.]
      Optimum energy:  72636.0
      Nodes in/out:  4126 11234
    Updated center to [ 89 378  11]
Added appearance for "n0" in frame 5 with seed coordinates [  89.  378.   11.]
      Optimum energy:  73312.0
      Nodes in/out:  4283 11077
    

## adding others

In [11]:
others = []
for i in range(7):
    o = data.init_object("o"+str(i))
    objects.append(o)
    others.append(o)

In [12]:
oseeds = [( 70,205,12),
          (145,170,12),
          ( 50,160,12),
          (105,140,12),
          (190,220,12),
          (200,265,12),
          (220,335,20)]
for i in range(len(oseeds)):
    data.add_object_at( others[i], min_rs, max_rs, frame=0, seed=oseeds[i], segment_it=True )
    data.track(others[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "o0" in frame 0 with seed coordinates [  70.  205.   12.]
      Optimum energy:  79124.0
      Nodes in/out:  6668 8692
Added appearance for "o0" in frame 1 with seed coordinates [  70.  205.   12.]
      Optimum energy:  78408.0
      Nodes in/out:  6350 9010
    Updated center to [ 68 204  11]
Added appearance for "o0" in frame 2 with seed coordinates [  68.  204.   11.]
      Optimum energy:  78043.0
      Nodes in/out:  6171 9189
    Updated center to [ 67 204  11]
Added appearance for "o0" in frame 3 with seed coordinates [  67.  204.   11.]
      Optimum energy:  77007.0
      Nodes in/out:  6111 9249
    Updated center to [ 66 203  11]
Added appearance for "o0" in frame 4 with seed coordinates [  66.  203.   11.]
      Optimum energy:  75956.0
      Nodes in/out:  6086 9274
    Updated center to [ 65 203  11]
Added appearance for "o0" in frame 5 with seed coordinates [  65.  203.   11.]
      Optimum energy:  75110.0
      Nodes in/out:  6120 9240
    Update

# Save / Load 

In [13]:
data.save('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')

In [ ]:
data = Data4d( [], silent=False )
data.load('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')
objects = range(len(data.object_names))
ecells = [0,1]
neighbors = []

# Visualisation stuff 

In [15]:
w = data.show_frame(0,stackUnits=[1.,1.,5.],raise_window=True)
data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)

In [17]:
for f in range(len(data.images)):
    print 'Exporting frame ',f+1,'of',len(data.images)
    w = data.show_frame(f,stackUnits=[1.,1.,5.],raise_window=False)
    data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
    data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
    data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
    data.save_current_visualization( 'frame'+str(f)+'.jpg' )

Exporting frame  1 of 12
Exporting frame  2 of 12
Exporting frame  3 of 12
Exporting frame  4 of 12
Exporting frame  5 of 12
Exporting frame  6 of 12
Exporting frame  7 of 12
Exporting frame  8 of 12
Exporting frame  9 of 12
Exporting frame  10 of 12
Exporting frame  11 of 12
Exporting frame  12 of 12
